In [ ]:
from asapdiscovery.data.schema.ligand import Ligand

In [ ]:
input_sdf = "/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/combined_split_3d_20250113/130.sdf"

In [ ]:
ligand = Ligand.from_sdf(input_sdf)

In [ ]:
oemol = ligand.to_oemol()

In [ ]:
rdmol = ligand.to_rdkit()

In [ ]:
rdmol.GetNumConformers()

In [ ]:
rdmol.GetPropsAsDict()

In [ ]:
smiles = ligand.smiles

In [ ]:
true_smiles = smiles.split('.')[0]

In [ ]:
true_smiles

In [ ]:
import rdkit

In [ ]:
match = rdmol.GetSubstructMatches(rdkit.Chem.MolFromSmiles(true_smiles))
new_mol = rdkit.Chem.PathToSubmol(rdmol, match)

In [ ]:
new_moles = []
for frag in rdkit.Chem.GetMolFrags(rdmol):
    # create new molecule from original and fragment
    new_moles.append(rdkit.Chem.PathToSubmol(rdmol, frag))

In [ ]:
new_mol = [rdmol.GetAtomWithIdx(idx) for idx in frag]

In [ ]:
rdmol = ligand.to_rdkit()
editable = rdkit.Chem.EditableMol(rdmol)

In [ ]:
from rdkit.Chem.Draw import MolDrawing, DrawingOptions

In [ ]:
DrawingOptions.includeAtomNumbers=True

In [ ]:
_ = [new_rdmol.GetAtomWithIdx(idx) for idx in frag]

In [ ]:
rdmol = ligand.to_rdkit()
editable = rdkit.Chem.EditableMol(rdmol)
for i, idx in enumerate(frag):
    idx = idx - i
    new_rdmol = editable.GetMol()
    smiles = rdkit.Chem.MolToSmiles(new_rdmol)
    print(smiles)
    print(len(smiles))
    editable.RemoveAtom(idx)

In [ ]:
frag

In [ ]:
new_mol = editable.GetMol()
rdkit.Chem.rdmolops.SanitizeMol(new_mol, sanitizeOps=rdkit.Chem.rdmolops.SANITIZE_ALL)
sdf_str = rdkit.Chem.rdmolfiles.MolToMolBlock(new_mol)

In [ ]:
from asapdiscovery.data.backend.rdkit import rdkit_mol_to_sdf_str

In [ ]:
new_lig = Ligand.from_sdf_str(sdf_str, compound_name=ligand.compound_name)

In [ ]:
new_lig.to_sdf('test.sdf')

In [ ]:
# Define the input and output filenames
from openeye.oechem import oemolistream, oemolostream, OEWriteMolecule
output_sdf = 'output_ligand.sdf'

# Open the input SDF file
ifs = oemolistream(input_sdf)

# Create an output stream to write the ligand
ofs = oemolostream(output_sdf)

# Loop through the molecules in the input SDF file
ligand_count = 0
for mol in ifs.GetOEGraphMols():
    # Only process the first ligand
    if ligand_count == 0:
        # Write the first ligand to the output file
        OEWriteMolecule(ofs, mol)
        ligand_count += 1
        break

# Close the input and output streams
ifs.close()
ofs.close()

print(f"First ligand saved to: {output_sdf}")